In [1]:
import time
from datetime import datetime
import dask.dataframe as dd
from distributed import LocalCluster, Client
import spatialpandas as spd
from spatialpandas.geometry import (
    PointArray, MultiPointArray, LineArray,
    MultiLineArray, PolygonArray, MultiPolygonArray
)
from spatialpandas import GeoSeries, GeoDataFrame
%matplotlib inline

In [2]:
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=4,
                       threads_per_worker=2,
                       memory_limit='3 GB')


client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:46665 Dashboard: http://127.0.0.1:8790/status,Cluster Workers: 4 Cores: 8 Memory: 12.00 GB


In [3]:
cont_us_path = "/home/kcpevey/scipy/git/data/contiguous_us.parquet"
ddf = dd.read_parquet(cont_us_path)

In [4]:
df = ddf.map_partitions(
    lambda df: GeoDataFrame(dict(
        position=PointArray(df[['longitude', 'latitude']]),
        **{col: df[col] for col in df.columns}
    ))
)

In [5]:
# df = df.partitions[:10]

In [6]:
savepath = '/work/spd-scipy2020/us_cont_spatiallysorted.parquet'

In [7]:
t0 = time.time()
df.pack_partitions(npartitions=df.npartitions, shuffle='disk').to_parquet(savepath)
dt = time.time() - t0

In [8]:
with open(f'spatial_sort_time-{datetime.now()}.csv', 'w') as f:
    f.write(f'time_min,npartitions\n{dt/60},{df.npartitions}')

In [9]:
df = spd.io.read_parquet_dask(savepath)
df.head()

,position,latitude,longitude
hilbert_distance,,,
25629,"Point([-124.443, 24.447])",24.447,-124.443
99850,"Point([-124.532, 24.704])",24.704,-124.532
216826,"Point([-124.248, 24.591])",24.591,-124.248
253418,"Point([-124.085, 24.463])",24.463,-124.085
290728,"Point([-123.719, 24.554])",24.554,-123.719
